### Sumiran Rai
### Regd. No. 0 24040208007
### Deep Learning Lab 1

### List

In [1]:
a = [1.0, 2.0, 1.0]

In [2]:
a[0]

1.0

In [3]:
a[2] = 3.0

In [4]:
a

[1.0, 2.0, 3.0]

### Constructing our first tensors

In [5]:
import torch
a = torch.ones(3) # Creates a one-dimensional tensor of size 3 filled with 1s

C:\Users\sumir\AppData\Roaming\Python\Python312\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [6]:
a[1]

tensor(1.)

In [7]:
float(a[1])

1.0

In [8]:
a[2] = 2.0
a

tensor([1., 1., 2.])

### Essence of tensors

##### PyTorch tensors or NumPy arrays are contiguous memor blocks containing unboxec C numeric types rather than Python objects. Each element is a 32-bit float.

In [9]:
points = torch.zeros(6) # Using.zeros is just a way to get an appropriately sized array.
points[0] = 4.0 # We overwrite those zeros with the values we actually want.
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

In [10]:
 # We can also pass a Python list to the constructor, to the same effect:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [11]:
# To get the coordinates of the first point, we do the following:
float(points[0]), float(points[1])

(4.0, 1.0)

In [12]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]]) #2D tensor
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [13]:
points.shape

torch.Size([3, 2])

In [14]:
# We could also use zeros or ones to initialize the tensor, providing the size as a tuple
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [15]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
# Now we can access an individual element in the tensor using two indices.
points[0, 1]

tensor(1.)

In [16]:
# We can also access the first element in the tensor as we did before to get the 2D coordinates of the first point
points[0]

tensor([4., 1.])

### Indexing tensors

In [17]:
some_list = list(range(6))
some_list[:] 
some_list[1:4] # From element 1 to element 4 inclusive
some_list[1:] # From element 1 to the end of list
some_list[:4] # From start of the list to element 4 exclusive
some_list[:-1] # From start of the list to one before the last element
some_list[1:4:2] # From element 1 to element 4 exclusive, in steps of 2

[1, 3]

In [18]:
points[1:] # All rows after the first; implicitly all columns

tensor([[5., 3.],
        [2., 1.]])

In [19]:
points[1:, :] # All rows after the first, all columns

tensor([[5., 3.],
        [2., 1.]])

In [20]:
points[1:, 0] # All rows after the first; first column

tensor([5., 2.])

In [21]:
points[None] # Adds a dimension of size 1, just like unsqueeze

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

### Named tensors

In [22]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns] Grayscale -> RGB
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [23]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns] Single Image -> Batch of images

So sometimes the RGB channels are in dimension 0, and sometimes they are in dimension 1. But we can generalize by counting from the end: they are always in dimension–3, the third from the end. The lazy, unweighted mean can thus be written as follows:

In [24]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

PyTorch will allow us to multiply things that are the
same shape, as well as shapes where one operand is of size 1 in a given dimension. It
 also appends leading dimensions of size 1 automatically. This is a feature called broad
casting. batch_t of shape (2, 3, 5, 5) is multiplied by unsqueezed_weights of shape (3,
 1, 1), resulting in a tensor of shape (2, 3, 5, 5), from which we can then sum the third
 dimension from the end (the three channels):

In [25]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [26]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [27]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

C:\Users\sumir\AppData\Local\Temp\ipykernel_37616\2371314847.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/core/TensorImpl.h:1938.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [28]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [29]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [30]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [31]:
gray_named = (img_named[..., :3] * weights_named).sum('channels')

RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.

In [ ]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

#### Managing a tensor's dtype attribute

In [ ]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [ ]:
short_points.dtype

torch.int16

In [ ]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [ ]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)


In [ ]:
points_64 = torch.rand(5, dtype=torch.double) # rand initializes the tensor elements to random numbers between 0 and 1
points_short = points_64.to(torch.short)
points_64 * points_short # works from PyTorch 1.3 onwards

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

### The tensor API

In [ ]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

In [ ]:
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [ ]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

### Indexing into storsge

In [ ]:
points = torch.tensor([[4.0,1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

C:\Users\sumir\AppData\Local\Temp\ipykernel_29756\3168638266.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [ ]:
points_storage = points.storage()
points_storage[0]

4.0

In [ ]:
points.storage()[1]

1.0

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

#### Modifying stored values: In-place operations

In [ ]:
a = torch.ones(3, 2)

In [ ]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

#### Views of another tensor's storage

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [ ]:
second_point.size()

torch.Size([2])

In [ ]:
second_point.shape

torch.Size([2])

In [ ]:
points.stride()

(2, 1)

In [ ]:
second_point = points[1]
second_point.size()

torch.Size([2])

In [ ]:
second_point.storage_offset()

2

In [ ]:
second_point.stride()

(1,)

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

#### Transposing without copying

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [ ]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [ ]:
id(points.storage()) == id(points_t.storage())

False

In [ ]:
points.stride()

(2, 1)

In [ ]:
points_t.stride()

(1, 2)

#### Transposing in higher dimensions

In [ ]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape

torch.Size([3, 4, 5])

In [ ]:
transpose_t.shape

torch.Size([5, 4, 3])

In [ ]:
some_t.stride()

(20, 5, 1)

In [ ]:
transpose_t.stride()

(1, 5, 20)

#### Contiguous tensors

In [ ]:
points.is_contiguous()


True

In [ ]:
points_t.is_contiguous()

False

In [ ]:
points = torch.tensor([[4.0,1.0], [5.0, 3.0], [2.0, 1.0]])
points_t =points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [ ]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [ ]:
points_t.stride()

(1, 2)

In [ ]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [ ]:
points_t_cont.stride()

(3, 1)

In [ ]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

#### In addition to dtype, a PyTorch Tensor also has the notion of device, which is where


In [ ]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
points_gpu = points.to(device='cuda')

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
points_gpu = points.to(device='cuda:0')

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
points = 2 * points
points_gpu = 2 * points.to(device='cuda')

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
points_gpu = points_gpu + 4

NameError: name 'points_gpu' is not defined

In [ ]:
points_cpu = points_gpu.to(device='cpu')

NameError: name 'points_gpu' is not defined

In [ ]:
points_gpu = points.cuda()
points_gpu = points.cuda(0)
points_cpu = points_gpu.cpu()

AssertionError: Torch not compiled with CUDA enabled

#### Numpy interoperability

In [ ]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [ ]:
points = torch.from_numpy(points_np)

####  Serializing tensors

In [ ]:
torch.save(points, '../data/p1ch3/ourpoints.t')

RuntimeError: Parent directory ../data/p1ch3 does not exist.

In [ ]:
with open('../data/p1ch3/ourpoints.t','wb') as f:
 torch.save(points, f)

FileNotFoundError: [Errno 2] No such file or directory: '../data/p1ch3/ourpoints.t'

In [ ]:
points = torch.load('../data/p1ch3/ourpoints.t')

FileNotFoundError: [Errno 2] No such file or directory: '../data/p1ch3/ourpoints.t'

In [ ]:
with open('../data/p1ch3/ourpoints.t','rb') as f:
 points = torch.load(f)


FileNotFoundError: [Errno 2] No such file or directory: '../data/p1ch3/ourpoints.t'

#### Serializing to HDF5 with h5py

In [ ]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import h5py
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

FileNotFoundError: [Errno 2] Unable to synchronously create file (unable to open file: name = '../data/p1ch3/ourpoints.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [ ]:
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'r')
dset = f['coords']
last_points = dset[-2:]

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '../data/p1ch3/ourpoints.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
 # In[63]:
last_points = torch.from_numpy(dset[-2:])
f.close()

NameError: name 'dset' is not defined